# 통계적 가중치 기반

In [2]:
import glob

#긍정리뷰 100개 불러오기
pos_review=(glob.glob("c:/workspace3/data/train/pos/*.txt"))[0:100]

lines_pos=[]

for i in pos_review:
    try:
        f = open(i, 'r')
        temp = f.readlines()[0]
        lines_pos.append(temp)
        f.close()
    except :
        continue

len(lines_pos)

100

In [3]:
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
from sklearn.feature_extraction.text import TfidfVectorizer

tokenizer = RegexpTokenizer('[\w]+')
stop_words = stopwords.words('english')

#TF-IDF 가중치 할당
vec = TfidfVectorizer(stop_words=stop_words)
vector_lines_pos = vec.fit_transform(lines_pos)
A=vector_lines_pos.toarray()

print(A.shape)
print(A)

#x축 단어, y축 문서

(100, 4001)
[[0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 ...
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.06538462 0.         ... 0.         0.         0.        ]
 [0.         0.23078109 0.         ... 0.         0.         0.        ]]


In [4]:
#현재 상태는 100개의 문서의 유사도
#단어간의 유사도를 구하는 것이 목적이므로
#단어-문서 행렬로 변경
#x축 문서, y축 단어

A=A.transpose()

print(A.shape)
print(A)

(4001, 100)
[[0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.06538462 0.23078109]
 [0.         0.         0.         ... 0.         0.         0.        ]
 ...
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]]


In [5]:
import numpy as np
from scipy import sparse

#밀집행렬(dense array)
a=np.array([[0.5,0,0],[0,1,0],[0.7,0,1.5]])

#밀집행렬을 희소행렬(sparse array)로 변환
#밀집행렬의 단점: 0이 많을 경우 메모리 낭비가 될 수 있음
#희소행렬은 0이 아닌 값들의 위치와 값만 기록하여 메모리를 절약하는 방식
b=sparse.csr_matrix(a)  
print(b)

# (0,0) 0.5 => 인덱스 0,0에 값 0.5
# (1,1) 1 => 인덱스 1,1에 값 1
# (2,0) 0.7 => 인덱스 2,0에 값 0.7
# (2,2) 1 => 인덱스 2,2에 값 1.5

c=b.toarray() #희소행렬을 밀집행렬로 변환
print(c)

  (0, 0)	0.5
  (1, 1)	1.0
  (2, 0)	0.7
  (2, 2)	1.5
[[0.5 0.  0. ]
 [0.  1.  0. ]
 [0.7 0.  1.5]]


In [6]:
from sklearn.metrics.pairwise import cosine_similarity

A_sparse = sparse.csr_matrix(A)

#코사인 유사도 계산
similarities_sparse = cosine_similarity(A_sparse,dense_output=False)

# todok() 행렬을 딕셔너리 형태로 변환
list(similarities_sparse.todok().items())[35000:35010]

#list(similarities_sparse.todok().items())[-10:]
#단어 이름이 아닌 인덱스 형태로 출력됨

[((108, 1469), 0.37803585968894865),
 ((108, 1470), 0.2189685434746738),
 ((108, 1476), 0.06407477897013734),
 ((108, 1477), 0.185189577514238),
 ((108, 1480), 0.20111036876169444),
 ((108, 1489), 0.06995711757772019),
 ((108, 1496), 0.10714874067068783),
 ((108, 1503), 0.30487333830091773),
 ((108, 1504), 0.30487333830091773),
 ((108, 1512), 0.30487333830091773)]

In [11]:
# (1469,108), 0.37 1469 단어와 108 단어의 유사도는 37%
vec.get_feature_names_out()[1469]

'fraud'

In [8]:
import pandas as pd
import numpy as np

#위의 결과값을 데이터프레임으로 출력
df=pd.DataFrame(list(similarities_sparse.todok().items()),columns=["words","weight"])

df2=df.sort_values(by=['weight'],ascending=False)
df2=df2.reset_index(drop=True)

#단어 자신끼리의 짝은 1이 되므로 1보다 작은 항목들만 출력시킴
df3=df2.loc[np.round(df2['weight']) < 1]
df3=df3.reset_index(drop=True)
df3.head(10)

,words,weight
0,"(3372, 3971)",0.499961
1,"(3971, 3372)",0.499961
2,"(2554, 1192)",0.499958
3,"(1192, 2554)",0.499958
4,"(1321, 2468)",0.499957
5,"(710, 2468)",0.499957
6,"(2468, 710)",0.499957
7,"(2468, 1321)",0.499957
8,"(889, 2146)",0.499909
9,"(2146, 889)",0.499909


In [9]:
for i,row in enumerate(df3.iterrows()):
#                            개별행
    a=vec.get_feature_names_out()[row[1][0][0]]
    b=vec.get_feature_names_out()[row[1][0][1]]

    print(f'{a},{b}=>{row[1][1]:.2f}')
    
    if i>10:
        break

stop,writers=>0.50
writers,stop=>0.50
past,essence=>0.50
essence,past=>0.50
farewell,older=>0.50
concubine,older=>0.50
older,concubine=>0.50
older,farewell=>0.50
deeply,made=>0.50
made,deeply=>0.50
watch,classes=>0.50
classes,watch=>0.50


In [ ]:
# sparse array 희소행렬
#               [1,1,1,1,1,1,0]

#   dense array 밀집행렬
#               [0,0,0,0,0,0,1] - 원핫인코딩

# sparse array 희소표현
#               (0,6) 1

#   dense array 밀집표현
#               [0,0,0,0,0,0,1]